In [18]:
tsv = {"genre": [], "filename": [], "year": [], "id": [], "score": [], "sentence1": [], "sentence2": []}

with open("sts-train.tsv", "r", encoding='utf-8') as f:
    for line in f.readlines()[1:]:
        line = line.split("\t")
        
        for i, key in enumerate(tsv.keys()):
            tsv[key].append(line[i].replace("\n", ""))

In [19]:
for key in tsv.keys():
    print(len(tsv[key]))

5749
5749
5749
5749
5749
5749
5749


In [20]:
import pandas as pd

df = pd.DataFrame(tsv)
df.head()

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,0001,5.000,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,main-captions,MSRvid,2012test,0004,3.800,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,main-captions,MSRvid,2012test,0005,3.800,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,main-captions,MSRvid,2012test,0006,2.600,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,main-captions,MSRvid,2012test,0009,4.250,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.


In [1]:
def get_tsv_data(tsv_dir: str):
    '''
    tsv_dir : directory of tsv file.
    return : dictionary of data.
    '''
    tsv = {"genre": [], "filename": [], "year": [], "id": [], "score": [], "sentence1": [], "sentence2": []}

    with open(tsv_dir, "r", encoding='utf-8') as f:
        for line in f.readlines()[1:]:
            line = line.split("\t")

            for i, key in enumerate(tsv.keys()):
                tsv[key].append(line[i].replace("\n", ""))
    
    return tsv

train_tsv = get_tsv_data("sts-train.tsv")
valid_tsv = get_tsv_data("sts-dev.tsv")
test_tsv = get_tsv_data("sts-test.tsv")

In [2]:
from transformers import AutoTokenizer
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
print(tokenizer.pad_token_id)

def prepare_data(tsv_dict):
    label = tsv_dict["score"]
    sentence1 = tsv_dict["sentence1"]
    sentence2 = tsv_dict["sentence2"]
    x = [(tokenizer.encode(s1), tokenizer.encode(s2)) for s1, s2 in tqdm(zip(sentence1, sentence2))]
    assert len(x) == len(label)
    return x, label

train_data = prepare_data(train_tsv)
valid_data = prepare_data(valid_tsv)
test_data = prepare_data(test_tsv)

c:\Users\rando\Anaconda3\envs\boost\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
5749it [00:02, 2161.17it/s]
1500it [00:00, 2050.99it/s]
1379it [00:00, 2151.56it/s]


In [4]:
train_x, train_y = train_data
valid_x, valid_y = valid_data
test_x, test_y = test_data

In [5]:
import numpy as np

np.save("train_x.npy", np.array(train_x, dtype='object'))
np.save("train_y.npy", np.array(train_y, dtype='object'))

np.save("valid_x.npy", np.array(valid_x, dtype='object'))
np.save("valid_y.npy", np.array(valid_y, dtype='object'))

np.save("test_x.npy", np.array(test_x, dtype='object'))
np.save("test_y.npy", np.array(test_y, dtype='object'))

In [62]:
train_x = np.load("train_x.npy", allow_pickle=True)

train_x[1]

array([list([0, 12279, 2]),
       list([0, 1078, 2470, 2747, 2259, 5227, 3692, 3720, 5255, 16, 19873, 7933, 4699, 2218, 4640, 16, 4543, 10131, 2170, 4249, 4167, 5029, 2496, 31369, 9482, 2116, 597, 13177, 575, 2052, 7245, 1041, 2371, 2062, 18, 2])],
      dtype=object)

위와 같이 sentence1과 sentence2의 길이가 크게 차이나는 경우가 있어 bucketing이 필요한 상황.